# Google Scholar Webscrape

Code accessing Google Scholar for a given author and SerpAPI API key.


Outputs (to mounted Drive, currently ~/Colab Notebooks/wangweb):

1.   'pubs.txt': Results of scrape formatted as dictionary
2.   'web_pub.txt': Reformatted results for website

Currently written for Prof. Rui Wang group by Takashi Yokokura, UC Berkeley.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Colab command to navigate to the folder holding the homework,
# CHANGE FOR YOUR SPECIFIC FOLDER LOCATION IN GOOGLE DRIVE
# Note: if there are spaces in the path, you need to preceed them with a backslash '\'
%cd /content/drive/My\ Drive/Colab\ Notebooks/wangweb

import string
###
"""
API KEY is free version - 100 pulls/month only
Login is using tyokokur github account (linked to berkeley.edu email)
"""
###

!pip install google-search-results
def scholar_sync(pubs_filename):
  from serpapi import GoogleSearch

  params = {
    "engine": "google_scholar_author",
    "hl": "en", # English
    "author_id": "IdNvgY0AAAAJ&hl", # Rui Wang
    "api_key": "5f07e581fedb60c9bbf2bfb6f7624870b14b01500702049975fd56e52d6cc018", # API key for tyokokur
    "num": 1000 # Max number of publications
  }

  search = GoogleSearch(params)
  results = search.get_dict() # Output
  arts = results['articles']
  with open(pubs_filename, 'w') as output_file:
    output_file.write('[')
    for i in results['articles']: output_file.write(str(i)+', ')
    output_file.write(']')
  return

scholar_sync('pubs.txt')
## Output file and change locally if needed

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/wangweb
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=18c8390ea2e3ca8e0e8a8dfcca65fd6996dc388b3391c41d5b2a97ddba21397e
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [2]:
with open('pubs.txt', 'r') as input_file:
  arts = input_file.read()
arts = eval(arts)
sort_arts = sorted(arts, reverse=True, key=lambda d: d['year']) # Sort by descending pub dat
sort_arts
for i in sort_arts:
  if '...' in i['authors']:
    i['authors'] = i['authors'][:-len('...')] + 'et. al' # et al. for entries with too many authors

  if '(' in i['publication'].split(' ')[-3]: # Issue number included
    i.update({'journal':' '.join(i['publication'].split(' ')[:-4])})
    i.update({'pages': ' '.join(i['publication'].split(' ')[-4:-1])[:-1]}) # [:-1] Get rid of final comma
  elif '-' not in i['publication'][-2]: # Online pub with no page number or issue
    i.update({'journal':' '.join(i['publication'].split(' ')[:-3])})
    i.update({'pages': ' '.join(i['publication'].split(' ')[-3:-1])[:-1]}) # [:-1] Get rid of final comma

In [3]:
INTRO = """
  <section id="two" class="wrapper style2">
				<div class="container">
					<div class = "inner">
						<div class="content">
              <ol reversed>
"""
TEMPLATE = """
<li>{authors}. {year}. "{title}". <em>{journal}</em>. {pages}. <a href="{link}">Link</a></li>
"""

In [4]:
with open('web_pub.txt', 'w') as output_file:
  for i in sort_arts:
    output_file.write(TEMPLATE.format(**i))